<a href="https://colab.research.google.com/github/Renzou1/treinamento-h2ia/blob/main/05_minimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usando busca adversarial para jogar Damas

In [1]:
import numpy as np
from collections import deque
import math

In [2]:
X_PIECE = True
Z_PIECE = False
X_TURN = True
Z_TURN = False

In [3]:
def create_board():
  board = np.array([
    ['( )', '(x)', '( )', '( )', '( )', '(z)', '( )', '(z)'],
    ['(x)', '( )', '(x)', '( )', '( )', '( )', '(z)', '( )'],
    ['( )', '(x)', '( )', '( )', '( )', '(z)', '( )', '(z)'],
    ['(x)', '( )', '(x)', '( )', '( )', '( )', '(z)', '( )'],
    ['( )', '(x)', '( )', '( )', '( )', '(z)', '( )', '(z)'],
    ['(x)', '( )', '(x)', '( )', '( )', '( )', '(z)', '( )'],
    ['( )', '(x)', '( )', '( )', '( )', '(z)', '( )', '(z)'],
    ['(x)', '( )', '(x)', '( )', '( )', '( )', '(z)', '( )'],
  ])
  return board

def print_board(state):
  i = 0
  j = 0
  print("   ", end="")
  while j < 8:
    print(j, end="  ")
    j = j + 1

  j = 0
  print()
  while j < 8:
    print(j, end=" ")
    while i < 8:
      print(state.board[i][j], end = "")
      i = i + 1
    print()
    j = j + 1
    i = 0


def inBounds(x, y):
  return x >= 0 and x <= 7 and y >= 0 and y <= 7

def moveable(x, y, current_state):
  if inBounds(x, y):
    if current_state.board[x][y] == '( )':
      return True
  return False

In [4]:
class GameState:
  def __init__(self, turn):
    self.board = create_board()
    self.turn = turn
    self.pieces = self.create_pieces()
    self.x_pieces = 12
    self.z_pieces = 12

  def passTurn(self):
    self.turn = not self.turn

  def create_pieces(self):
    pieces = np.array([])
    xs = np.argwhere(self.board == '(x)')
    zs = np.argwhere(self.board == '(z)')
    i = 0
    while i < len(xs):
      pieces = np.append(pieces, Piece(xs[i][0], xs[i][1], X_PIECE))
      i = i + 1
    i = 0
    while i < len(zs):
      pieces = np.append(pieces, Piece(zs[i][0], zs[i][1], Z_PIECE))
      i = i + 1

    return pieces

  def get_piece(self, x, y):
    for piece in self.pieces:
      if piece.x == x and piece.y == y:
        return piece
    print("failed to locate piece at", x, y)

  def calculate_value(self):
    return self.x_pieces - self.z_pieces

In [5]:
def moveLeft(current_state, piece_x, piece_y):
  #already checked bounds
  piece = current_state.get_piece(piece_x, piece_y)
  current_state.board[piece_x][piece_y] = '( )'

  if piece.side == X_PIECE:

    if current_state.board[piece_x - 1][piece_y + 1] == '(z)':
      current_state.board[piece_x - 1][piece_y + 1] = '( )'
      current_state.get_piece(piece_x - 1, piece_y + 1).kill(current_state)
      piece.x = piece.x - 2
      piece.y = piece.y + 2

    else:
      piece.x = piece.x - 1
      piece.y = piece.y + 1

    current_state.board[piece.x][piece.y] = '(x)'

  elif piece.side == Z_PIECE:

    if current_state.board[piece_x - 1][piece_y - 1] == '(x)':
      current_state.board[piece_x - 1][piece_y - 1] = '( )'
      current_state.get_piece(piece_x - 1, piece_y - 1).kill(current_state)
      piece.x = piece.x - 2
      piece.y = piece.y - 2

    else:
      piece.x = piece.x - 1
      piece.y = piece.y - 1

    current_state.board[piece.x][piece.y] = '(z)'

  current_state.passTurn()
  return(current_state)


def moveRight(current_state, piece_x, piece_y):
  #already checked bounds
  piece = current_state.get_piece(piece_x, piece_y)
  current_state.board[piece.x][piece.y] = '( )'

  if piece.side == X_PIECE:

    if current_state.board[piece.x + 1][piece.y + 1] == '(z)':
      current_state.board[piece.x + 1][piece.y + 1] = '( )'
      current_state.get_piece(piece.x + 1, piece.y + 1).kill(current_state)
      piece.x = piece.x + 2
      piece.y = piece.y + 2

    else:
      piece.x = piece.x + 1
      piece.y = piece.y + 1

    current_state.board[piece.x][piece.y] = '(x)'

  elif piece.side == Z_PIECE:

    if current_state.board[piece.x + 1][piece.y - 1] == '(x)':
      current_state.board[piece.x + 1][piece.y - 1] = '( )'
      current_state.get_piece(piece.x + 1, piece.y - 1).kill(current_state)
      piece.x = piece.x + 2
      piece.y = piece.y - 2

    else:
      piece.x = piece.x + 1
      piece.y = piece.y - 1

    current_state.board[piece.x][piece.y] = '(z)'

  current_state.passTurn()
  return current_state



class Piece:
  def __init__(self, x, y, x_or_z, queen=False, alive=True):
    self.x = x
    self.y = y
    self.side = x_or_z
    self.queen = queen
    self.alive = alive

  def getPosition(self):
    return self.x, self.y

  def kill(self, current_state):
    self.alive = False
    if self.side == X_PIECE:
      current_state.x_pieces = current_state.x_pieces - 1

    else:
      current_state.z_pieces = current_state.z_pieces - 1


  def checkPieceMoves(self, current_state):
    moves = np.array([])
    if self.alive == False or self.side != current_state.turn:
      return moves
    else:
      if self.canMoveLeft(current_state):
        #if moveLeft not in moves:
          moves = np.append(moves, moveLeft)

      if self.canMoveRight(current_state):
        #if moveRight not in moves:
          moves = np.append(moves, moveRight)
    return moves

    #if


  def canCaptureLeft(self, current_state):
    if self.side == X_PIECE:
      if inBounds(self.x - 1, self.y + 1) and inBounds(self.x - 2, self.y + 2):
        if current_state.board[self.x - 1][self.y + 1] == '(z)' and current_state.board[self.x - 2][self.y + 2] == '( )':
          return True
    elif self.side == Z_PIECE:
      if inBounds(self.x - 1, self.y - 1) and inBounds(self.x - 2, self.y - 2):
        if current_state.board[self.x - 1][self.y - 1] == '(x)' and current_state.board[self.x - 2][self.y - 2] == '( )':
          return True
    return False

  def canCaptureRight(self, current_state):
    if self.side == X_PIECE:
      if inBounds(self.x + 1, self.y + 1) and inBounds(self.x + 2, self.y + 2):
        if current_state.board[self.x + 1][self.y + 1] == '(z)' and current_state.board[self.x + 2][self.y + 2] == '( )':
          return True
    elif self.side == Z_PIECE:
      if inBounds(self.x + 1, self.y - 1) and inBounds(self.x + 2, self.y - 2):
        if current_state.board[self.x + 1][self.y - 1] == '(x)' and current_state.board[self.x + 2][self.y - 2] == '( )':
          return True
    return False

  def canMoveLeft(self, current_state):
    if self.side == current_state.turn:
      if self.side == X_PIECE:
        return self.canCaptureLeft(current_state) or moveable(self.x - 1, self.y + 1, current_state)

      if self.side == Z_PIECE:
        return self.canCaptureLeft(current_state) or moveable(self.x - 1, self.y - 1, current_state)

    return False

  def canMoveRight(self, current_state):
    if self.side == current_state.turn:
      if self.side == X_PIECE:
        return self.canCaptureRight(current_state) or moveable(self.x + 1, self.y + 1, current_state)

      if self.side == Z_PIECE:
        return self.canCaptureRight(current_state) or moveable(self.x + 1, self.y - 1, current_state)

    return False

In [6]:
MINI = True
MAX = False

class MaxNode:
  def __init__(self, piece_x=-1, piece_y=-1, state=None,parent=None, move=None, value=0):
    self.parent = parent
    self.move = move
    self.state = state
    self.value = value
    self.favorite_child = None
    self.player = MAX
    self.piece_x = piece_x
    self.piece_y = piece_y

    if parent == None:
      self.value = -math.inf

  def add_child(self, move, piece_x, piece_y):

    child_state = copy.deepcopy(self.state)
    move(child_state, piece_x, piece_y)
    child_value = child_state.calculate_value()
    child = MiniNode(piece_x=piece_x,piece_y=piece_y,state=copy.deepcopy(child_state), parent=self, move=move, value=child_value)
    child.pass_values_upward() # picks highest min
    if child_state != None:
      return child #might be wrong
    else:
      return None

  def set_initial_state(self, state):
    self.state = np.array(state)

  def set_value(self, value):
    self.value = value

  def pass_values_upward(self):
    if self.parent != None:
      if self.parent.player == MINI:
        if self.value < self.parent.value:
          self.parent.value = self.value
          self.parent.favorite_child = self
      else:
        if self.value > self.parent.value:
          self.parent.value = self.value
          self.parent.favorite_child = self

In [10]:
MINI = True
MAX = False

class MiniNode:
  def __init__(self, piece_x=-1, piece_y=-1, state=None,parent=None, move=None, value=0):
    self.parent = parent
    self.move = move
    self.state = state
    self.value = value
    self.favorite_child = None
    self.player = MINI
    self.piece_x = piece_x
    self.piece_y = piece_y

    if parent == None:
      self.value = math.inf

  def add_child(self, move, piece_x, piece_y):
    child_state = copy.deepcopy(self.state)
    move(child_state, piece_x, piece_y)
    child_value = child_state.calculate_value()
    child = MaxNode(piece_x=piece_x,piece_y=piece_y,state=copy.deepcopy(child_state), parent=self, move=move, value=child_value)
    child.pass_values_upward() # picks lowest max
    if child_state != None:
      return child #might be wrong
    else:
      return None

  def set_initial_state(self, state):
    self.state = np.array(state)

  def set_value(self, value):
    self.value = value

  def pass_values_upward(self):
    if self.parent != None:
      if self.parent.player == MINI:
        if self.value < self.parent.value:
          self.parent.value = self.value
          self.parent.favorite_child = self
      else:
        if self.value > self.parent.value:
          self.parent.value = self.value
          self.parent.favorite_child = self

In [8]:
from ast import Continue
import copy

CUT = False
FAIL = True

MINI = True
MAX = False

CUT = False
FAIL = True

MINI = True
MAX = False

def minimax(mainState, max_depth=3):
  imaginaryState = copy.deepcopy(mainState)
  root = MaxNode(state=imaginaryState)
  minimax_recursive(root, max_depth, imaginaryState)

  return root

def minimax_recursive(node, max_depth, state):
  limit_reached = False
  if max_depth == 0:
    return CUT
  else:
    for piece in state.pieces:
      if piece.side == state.turn and piece.alive == True:
        moves = piece.checkPieceMoves(state)
        if moves.size != 0:
          for move in moves:
            child = node.add_child(move, piece.x, piece.y)
            if check_alpha_beta(node, child):
              return
            else:
              result = minimax_recursive(child, max_depth - 1, child.state)
              if result == None: #alpha beta
                continue
              elif result == CUT:
                limit_reached = True
              elif result != FAIL:
                return result
          if limit_reached:
            return CUT
          else:
            return FAIL

def check_alpha_beta(node, child):
  #return False
  if node.parent != None:
    if (node.parent.player == MAX and child.value < node.parent.value) or (node.parent.player == MINI and child.value > node.parent.value):
      return True
  return False

In [11]:
def canMoveRight(mainState, piece_x, piece_y):
   return mainState.get_piece(piece_x, piece_y).canMoveRight(mainState)

def canMoveLeft(mainState, piece_x, piece_y):
  return mainState.get_piece(piece_x, piece_y).canMoveLeft(mainState)

def get_player_move():
    piece_x, piece_y, direction = input().split()
    piece_x = int(piece_x)
    piece_y = int(piece_y)
    return piece_x, piece_y, direction

def AI_move(mainState):
  print()
  print_board(mainState)
  print()
  print("AI turn:")
  root = minimax(mainState, max_depth=50)
  child = root.favorite_child
  child.move(mainState, child.piece_x, child.piece_y)

def play_game():
  mainState = GameState(Z_TURN)

  while 1 == 1:
    print_board(mainState)
    print()
    print("Your Turn:")
    print("Type: piece_x piece_y right/left")
    piece_x, piece_y, direction = get_player_move()
    while mainState.get_piece(piece_x, piece_y) == None or mainState.get_piece(piece_x, piece_y).side != mainState.turn:
       print("Incorrect position")
       piece_x, piece_y, direction = get_player_move()

    if direction == 'right':
        if canMoveRight(mainState, piece_x, piece_y):
            moveRight(mainState, piece_x, piece_y)
    elif direction == 'left':
        if canMoveLeft(mainState, piece_x, piece_y):
            moveLeft(mainState, piece_x, piece_y)

    AI_move(mainState)

play_game()

   0  1  2  3  4  5  6  7  
0 ( )(x)( )(x)( )(x)( )(x)
1 (x)( )(x)( )(x)( )(x)( )
2 ( )(x)( )(x)( )(x)( )(x)
3 ( )( )( )( )( )( )( )( )
4 ( )( )( )( )( )( )( )( )
5 (z)( )(z)( )(z)( )(z)( )
6 ( )(z)( )(z)( )(z)( )(z)
7 (z)( )(z)( )(z)( )(z)( )

Your Turn:
Type: piece_x piece_y right/left
2 5 left

   0  1  2  3  4  5  6  7  
0 ( )(x)( )(x)( )(x)( )(x)
1 (x)( )(x)( )(x)( )(x)( )
2 ( )(x)( )(x)( )(x)( )(x)
3 ( )( )( )( )( )( )( )( )
4 ( )(z)( )( )( )( )( )( )
5 (z)( )( )( )(z)( )(z)( )
6 ( )(z)( )(z)( )(z)( )(z)
7 (z)( )(z)( )(z)( )(z)( )

AI turn:
   0  1  2  3  4  5  6  7  
0 ( )(x)( )(x)( )(x)( )(x)
1 (x)( )(x)( )(x)( )(x)( )
2 ( )( )( )(x)( )(x)( )(x)
3 (x)( )( )( )( )( )( )( )
4 ( )(z)( )( )( )( )( )( )
5 (z)( )( )( )(z)( )(z)( )
6 ( )(z)( )(z)( )(z)( )(z)
7 (z)( )(z)( )(z)( )(z)( )

Your Turn:
Type: piece_x piece_y right/left
1 4 right

   0  1  2  3  4  5  6  7  
0 ( )(x)( )(x)( )(x)( )(x)
1 (x)( )(x)( )(x)( )(x)( )
2 ( )( )( )(x)( )(x)( )(x)
3 (x)( )(z)( )( )( )( )( )
4 ( )( )( )

KeyboardInterrupt: Interrupted by user